In [1]:
import pandas as pd

df = pd.read_pickle("organized_results.pickle")
df

,着順,枠番,馬番,馬名,斤量,騎手,単勝,人気,性別,年齢,体重,体重変化
201901010101,1,1,1,ゴルコンダ,54.0,ルメール,1.4,1,1,2,518,-16
201901010101,2,3,3,プントファイヤー,54.0,岩田康誠,3.5,2,1,2,496,-8
201901010101,3,4,4,ラグリマスネグラス,51.0,団野大成,46.6,6,1,2,546,6
201901010101,4,8,9,キタノコドウ,51.0,菅原明良,56.8,7,1,2,458,-8
201901010101,5,5,5,ネモフィラブルー,54.0,川島信二,140.3,9,1,2,436,0
...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,52.0,大野拓弥,45.7,10,0,3,414,6
201910020812,11,3,3,プリヴェット,55.0,藤岡康太,29.8,9,0,4,486,2
201910020812,12,5,7,アドマイヤクィーン,55.0,松山弘平,29.6,8,0,4,476,6
201910020812,13,8,13,タムロドリーム,49.0,亀田温心,134.8,12,0,3,472,2


In [2]:
clip_rank = lambda x: x if x < 4 else 4

df["rank"]=df["着順"].map(clip_rank)
# df["着順"].value_counts()

In [3]:
df.drop(["着順","馬名"],axis=1,inplace=True)

In [4]:
df

,枠番,馬番,斤量,騎手,単勝,人気,性別,年齢,体重,体重変化,rank
201901010101,1,1,54.0,ルメール,1.4,1,1,2,518,-16,1
201901010101,3,3,54.0,岩田康誠,3.5,2,1,2,496,-8,2
201901010101,4,4,51.0,団野大成,46.6,6,1,2,546,6,3
201901010101,8,9,51.0,菅原明良,56.8,7,1,2,458,-8,4
201901010101,5,5,54.0,川島信二,140.3,9,1,2,436,0,4
...,...,...,...,...,...,...,...,...,...,...,...
201910020812,7,12,52.0,大野拓弥,45.7,10,0,3,414,6,4
201910020812,3,3,55.0,藤岡康太,29.8,9,0,4,486,2,4
201910020812,5,7,55.0,松山弘平,29.6,8,0,4,476,6,4
201910020812,8,13,49.0,亀田温心,134.8,12,0,3,472,2,4


In [5]:
test = pd.get_dummies(df)
len(test.columns)

197

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


X = test.drop(["rank"],axis=1)
y = test["rank"]

X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=0)

In [7]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [8]:
print(model.score(X_train,y_train),model.score(X_test,y_test))

0.7791338193092984 0.7791288286901646


In [9]:
y_pred = model.predict(X_test)
y_pred

array([4, 4, 4, ..., 4, 4, 4], dtype=int64)

In [10]:
y_train.value_counts()

4    23621
1     2235
3     2232
2     2229
Name: rank, dtype: int64

In [11]:
pd.Series(y_pred).value_counts()

4    12994
dtype: int64

In [17]:
from imblearn.under_sampling import RandomUnderSampler


rank_1 = y_train.value_counts()[1]
rank_2 = y_train.value_counts()[2]
rank_3 = y_train.value_counts()[3]

rus = RandomUnderSampler(sampling_strategy={1: rank_1, 2: rank_2, 3: rank_3, 4: rank_1}, random_state=71)

X_train_rus,y_train_rus = rus.fit_sample(X_train,y_train)
pd.Series(y_train_rus).value_counts()

4    2235
1    2235
3    2232
2    2229
Name: rank, dtype: int64

In [18]:
model = LogisticRegression()
model.fit(X_train_rus,y_train_rus)
print(model.score(X_train,y_train),model.score(X_test,y_test))

0.5441831315763433 0.5435585654917654


In [19]:
y_pred = model.predict(X_test)
pd.Series(y_pred).value_counts()

4    6579
1    3417
3    2185
2     813
dtype: int64

In [23]:
pred_df = pd.DataFrame({"pred":y_pred,"actual":y_test})
pred_df

,pred,actual
201905010712,1,4
201904020203,1,1
201904030103,4,4
201906020103,4,4
201908020303,4,4
...,...,...
201907020608,3,4
201906020806,4,4
201901020511,3,1
201906040810,1,4


In [32]:
pred_df[pred_df["pred"]==1]["actual"].value_counts(1)

4    0.492830
1    0.195786
2    0.169154
3    0.142230
Name: actual, dtype: float64

In [33]:
pred_df[pred_df["pred"]==4]["actual"].value_counts(1)

4    0.932665
3    0.031008
2    0.023104
1    0.013224
Name: actual, dtype: float64

In [55]:
coefs = pd.Series(model.coef_[0],index=X.columns).sort_values()

In [56]:
# df[df["騎手"]=="岩田康誠"]["rank"].value_counts(1)
df[df["騎手"]=="ルメール"]["rank"].value_counts(1)

4    0.435484
1    0.250000
2    0.193548
3    0.120968
Name: rank, dtype: float64

In [57]:
X.columns

Index(['枠番', '馬番', '斤量', '単勝', '人気', '性別', '年齢', '体重', '体重変化', '騎手_アヴドゥ',
       ...
       '騎手_難波剛健', '騎手_高倉稜', '騎手_高松亮', '騎手_高田潤', '騎手_高野和馬', '騎手_鮫島克駿',
       '騎手_鮫島良太', '騎手_黒岩悠', '騎手_黛弘人', '騎手_Ｍ．デム'],
      dtype='object', length=196)

In [60]:
coefs[{'枠番', '馬番', '斤量', '単勝', '人気', '性別', '年齢', '体重', '体重変化'}].sort_values()

人気     -0.112149
年齢     -0.022386
単勝     -0.009822
枠番     -0.008046
体重      0.001276
性別      0.001536
体重変化    0.002321
馬番      0.003116
斤量      0.004164
dtype: float64

In [61]:
X_test2 = test.drop(["rank"],axis=1).head(10)
y_test2 = test["rank"].head(10)

In [70]:
pred_test2 = model.predict(X_test2)
pred_test2

array([1, 1, 4, 4, 4, 1, 4, 3, 2, 1], dtype=int64)

In [69]:
# print("coefficient = ", model.coef_)
# print("intercept = ", model.intercept_)
model.predict_proba(X_test2)

array([[0.41782312, 0.27922607, 0.22682517, 0.07612564],
       [0.37428434, 0.28679977, 0.24583264, 0.09308325],
       [0.18663473, 0.21078667, 0.29509145, 0.30748715],
       [0.12126513, 0.2151023 , 0.29478534, 0.36884723],
       [0.02502305, 0.09302762, 0.19884522, 0.6831041 ],
       [0.32541691, 0.2882314 , 0.27448419, 0.11186751],
       [0.04755368, 0.1276489 , 0.23358386, 0.59121357],
       [0.2213146 , 0.28305194, 0.29600131, 0.19963215],
       [0.27915505, 0.28641958, 0.28235998, 0.1520654 ],
       [0.30935904, 0.28501141, 0.27495993, 0.13066962]])

In [73]:
model.predict_proba(X_test2)[0]

array([0.41782312, 0.27922607, 0.22682517, 0.07612564])